# Intro to SqlAlchemy

### Introduction

SQLAlchemy is an object relational mapper library that relates our models to their corresponding tables and also will write some common SQL queries for us.  

Let's get started.

### Setting it up

The first step is to install the `flask_sqlalchemy` library which is located in the `requirements.txt` file and can be installed with a call to `pip3 install -r requirements.txt`.

* `app/__init__.py`

And if you look at the `app/__init__.py` file, you can see a `create_app` function that first sets the `SQLALCHEMY_DATABASE_URI` on our flask app, and then calls `db.init_app(app)` passing through our flask application. 

```python
def create_app():
    app = Flask(__name__)
    conn_str = "postgresql://localhost:5432/crm"
    app.config['SQLALCHEMY_DATABASE_URI'] = conn_str
    db.init_app(app)
    return app
```

The db is an instance of the SqlAlchemy class, so in these lines we are telling `SqlAlchemy` about a local postgres database called `crm`.

* `server.py`

Next, let's move onto the `server.py` file focusing on the first few lines.

```python
from app import create_app, db
from app.models import User

app = create_app()
```

So here, we can see we are importing the `create_app` function, and the `db` object we saw before.  We are also importing our User model.

Let's take a look at that next.

* `app/models/user.py`

```python
from app import db

class User(db.Model):
    __tablename__ = 'users'

    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
```

So in this file, notice that we are passing importing our db object (which has knowledge about the postgres database), and then having the User model inherit from `db.Model`.  We are then setting class variables of id, username and email.  And are setting them equal to `Column` instances that are specified as certain types like integer, or strings.  

> We also set attributes for username and email like specifying uniqueness, and that values cannot be null.  

### Reviewing with the Server file

Ok so back to our `server.py` file.

```python
from app import create_app, db
from app.models import User

app = create_app()
```

We can see that the create_app function connects the db object to our database, and that we import a User model that is connected to this database, as the model inherits from `db.Model`.

And then finally, in the last few lines we make this thing called `shell_context` that knows about both the db and User objects.

```python
@app.shell_context_processor
def make_shell_context():
    return {'db': db, 'User': User}
```

Let's see what this shell context is.

It turns out that sqlalchemy knows about our server.py file, because of the `.flaskenv` file that has the following:
```bash
export FLASK_APP=server.py
```

So if we run `flask shell`, it will see the make_shell_context function in the server.py file.  

Let's try it.

### Accessing the shell

We can access the shell by typing `flask shell`.

<img src="./flask-shell.png" width="70%">

And notice above that we have access to the `db` and `User` objects, as we imported them into our shell context with our line:

```python
@app.shell_context_processor
def make_shell_context():
    return {'db': db, 'User': User}
```

And from there, we can *almost* query the database with the following:

```python3
db.session.query(User).all()
```

If we try this, we'll get the following error.

```bash
port 5432 failed: FATAL:  database "crm" does not exist
```

So let's log into postgres, create the database crm, and then run the `create_tables.sql` file in the migrations folder to create the users table. 

> First connect to postgres.

```bash
psql
```

And then create the database called `crm`.

<img src="./build_db.png" width="70%">

And then we can exit out of postgres, and run the create table statement against the database.

```bash
psql -d crm -f migrations/create_tables.sql
```

### Getting there

Ok, so now if we log back into the shell.

```bash
flask shell
```

We can query the database.

<img src="./query-all.png" width="60%">

And if we want to see what sqlalchemy ran under the hood, we can do so with the following.

<img src="./last-statement.png" width="70%">

So we can see, we are selecting all of the records (and all columns) in the users table.  

And SqlAlchemy knew which columns and table to query by looking at the User model.

### Summary

In this lesson, we saw how to get up and running with sqlalchemy.  Some of the main steps were to:

1. Tell the sqlalchemy db object about the database

```python
db = SQLAlchemy()

def create_app():
    app = Flask(__name__)
    conn_str = "postgresql://localhost:5432/crm"
    app.config['SQLALCHEMY_DATABASE_URI'] = conn_str
    db.init_app(app)
    return app
```

2. Pass the db object into the respective model

```python
from app import db

class User(db.Model):
    __tablename__ = 'users'

    id = db.Column(db.Integer, primary_key=True)
```

3. User the .flaskenv file to set the FLASK_APP env variable

```
export FLASK_APP=server.py
```

4. In the `server.py` file, import the db object specify objects we want to access in the shell

```python
@app.shell_context_processor
def make_shell_context():
    return {'db': db, 'User': User}
```

5. Then, after creating the database and tables we can make queries
```bash
flask shell
```

```python
db.session.query(User).all()
```